### Data Cleaning

Cleaning the raw data scraped from Wikipedia. I will also add some geospatial information for further data analysis.

In [1]:
import pandas as pd
import numpy as np


In [6]:
df = pd.read_csv("C:/Users/User/Downloads/Documents/Largest WWTP/raw_largewwtp.csv")
df.head()


(21, 8)

In [8]:
df.dtypes
df.isna().sum()

Plant_Name          0
City                0
Country             0
Opening_Year        2
DryWeather(CMD)     6
WetWeather(CMD)    16
Area(sqkm)         14
Notes              26
dtype: int64

In [9]:
df =df.drop(columns =['Notes'])
df = df.fillna(0).copy()
df['Opening_Year'] = df['Opening_Year'].astype(int)

In [10]:
df[['Plant_Name','WetWeather(CMD)','DryWeather(CMD)','Area(sqkm)']] =\
df[['Plant_Name','WetWeather(CMD)','DryWeather(CMD)','Area(sqkm)']].replace(r'\[.*\]', '', regex=True) 
df['Area(sqkm)'] = df['Area(sqkm)'].astype(float)


In [11]:
df['DryWeather(CMD)'] = [float(str(val).replace(' ','').replace(',','')) for val in \
    df['DryWeather(CMD)'].values]

df['WetWeather(CMD)'] = [float(str(val).replace(' ','').replace(',','')) for val in \
    df['WetWeather(CMD)'].values]

df[['DryWeather(CMD)','WetWeather(CMD)']] = df[['DryWeather(CMD)','WetWeather(CMD)']]\
    .astype(int)

In [12]:
import pandas as pd
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapi")

In [15]:
def get_lat_lon(row):
    try:
        location = geolocator.geocode(f"{row['City']}, {row['Country']}")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None, None
df['Coordinates'] = df.apply(get_lat_lon, axis=1)
df[['Latitude', 'Longitude']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)
df.drop('Coordinates', axis=1, inplace=True)

,Plant_Name,City,Country,Opening_Year,DryWeather(CMD),WetWeather(CMD),Area(sqkm),Latitude,Longitude
0,Jean-R.-Marcotte Wastewater Treatment Plant,Montreal,Canada,1984,2780000,7600000,0.67,45.503182,-73.569806
1,Detroit Wastewater Treatment Plant,Detroit,USA,1940,2460000,6435000,0.53,42.331551,-83.046640
2,Stickney Water Reclamation Plant,Chicago,USA,1930,2665000,5450000,1.67,41.875562,-87.624421
3,Blue Plains Advanced Wastewater Treatment Plant,Washington D.C.,USA,1937,1450000,4073000,0.62,38.895037,-77.036543
4,Deer Island Waste Water Treatment Plant,Boston,USA,1968,1438000,4542000,0.60,42.355433,-71.060511
5,Abu Rawash Wastewater treatment plant,Giza,Egypt,2021,1600000,0,1.39,29.987075,31.211806
6,Atotonilco de Tula Plant,Mexico City,Mexico,2015,2000000,3000000,0.00,19.432630,-99.133178
7,Hyperion Water Reclamation Plant,Los Angeles,USA,1925,1041000,3000000,0.81,34.053691,-118.242766
8,Kuryanovo wastewater treatment facilities,Moscow,Russia,1950,2200000,0,0.00,55.625578,37.606392
9,Lyuberetskiye wastewater treatment facilities,Moscow,Russia,1963,3000000,0,0.00,55.625578,37.606392


In [16]:
df.to_csv(
    r'C:\Users\User\Downloads\Documents\Largest WWTP\clean_largewwtp.csv', index=False)

In [17]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point


In [18]:
data = pd.read_csv("C:/Users/User/Downloads/Documents/Largest WWTP/clean_largewwtp.csv")
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)
world = gpd.read_file("C:/Users/User/Downloads/Documents/Largest WWTP/world_shapefile/world-shapefiles.shp")
result = gpd.sjoin(gdf, world, how="inner", op="intersects")
result.to_file('new_shapefile.shp')

c:\Users\User\.conda\envs\VirtualEnvPython\Lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\User\AppData\Local\Temp\ipykernel_22796\1273322278.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: GEOGCS["GCS_unknown",DATUM["WGS_1984",SPHEROID["WG ...

  result = gpd.sjoin(gdf, world, how="inner", op="intersects")
C:\Users\User\AppData\Local\Temp\ipykernel_22796\1273322278.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result.to_file('new_shapefile.shp')
